In [1]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD


class Catcher(Environment):
    """Catcher
    Agent is a 3 pixel `basket` in the bottom of a square grid of size
    `grid_size`. Single pixels `fruits` fall from the top and return +1 reward
    if catch and -1 else.
    Attributes
    ----------
    grid_size : int
        Size of the square grid
    output_type: str
        Either give state description as raw 'pixels', or as the location of
        the fruit and basket 'position'. The 'pixels' state space size is
        2**(grid_size**2), while the 'position' state space size is
        grid_size**3.
    """
    def __init__(self, grid_size=10, output_shape=None, output_type='pixels'):
        self.grid_size = grid_size
        self.output_type = output_type
        self.output_shape = output_shape

        if output_shape is None:
            if output_type == 'pixels':
                output_shape = (grid_size**2, )
            elif output_type == 'position':
                output_shape = (3, )
        self.output_shape = output_shape
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self._state = out

    def _draw_state(self):
        """Convert state description into a square image
        """
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas

    def reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1
            else:
                return -1
        else:
            return 0

    def observe_image(self):
        canvas = self._draw_state()
        return canvas.reshape(1, 1, self.grid_size, self.grid_size)

    def observe(self):
        if self.output_type == 'pixels':
            canvas = self._draw_state()
            out = canvas.reshape((1, ) + self.output_shape)
        if self.output_type == 'position':
            out = self.state[0]
        return out

    def update(self, action):
        self._update_state(action)
        reward = self.reward()
        game_over = self.is_over
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self._state = np.asarray([0, n, m])[np.newaxis]

    @property
    def state(self):
        return self._state

    @property
    def is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    @property
    def description(self):
        return f"Catch game with grid size {self.grid_size}"


class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1]
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            targets[i] = model.predict(state_t)[0]
            Q_sa = np.max(model.predict(state_tp1)[0])
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets

   
# parameters
epsilon = .1  # exploration
num_actions = 3  # [move_left, stay, move_right]
epoch = 1000
max_memory = 500
hidden_size = 100
batch_size = 1
grid_size = 10

model = Sequential()
model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
model.add(Dense(hidden_size, activation='relu'))
model.add(Dense(num_actions))
model.compile(SGD(lr=.2), "mse")
    
# If you want to continue training from a previous model, just uncomment the line bellow
# model.load_weights("model.h5")

# Define environment/game
env = Catcher(grid_size)

# Initialize experience replay object
exp_replay = ExperienceReplay(max_memory=max_memory)

def train(model):
    # Train
    win_cnt = 0
    for e in range(1):
        loss = 0.
        env.reset()
        game_over = False
        # get initial input
        input_t = env.observe()

        while not game_over:
            input_tm1 = input_t
            # get next action
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, num_actions, size=1)
            else:
                q = model.predict(input_tm1)
                action = np.argmax(q[0])
                
            action = int(action)
            # apply action, get rewards and new state
            input_t, reward, game_over = env.act(action)
            if reward == 1:
                win_cnt += 1

            # store experience
            exp_replay.remember([input_tm1, action, reward, input_t], game_over)            
            
            # adapt model
            inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)
            
            display_screen(action,3000,inputs[0])            
            
            loss += model.train_on_batch(inputs, targets)
        print("Epoch {:03d}/999 | Loss {:.4f} | Win count {}".format(e, loss, win_cnt))

NameError: name 'Environment' is not defined

In [2]:
%matplotlib inline
import seaborn
seaborn.set()
import json
import matplotlib.pyplot as plt
import numpy as np
import time
from keras.models import model_from_json
from PIL import Image
from IPython import display
last_frame_time = 0
translate_action = ["Left","Stay","Right","Create Ball","End Test"]
grid_size = 10

def display_screen(action,points,input_t):
    global last_frame_time
    display.clear_output(wait=True)
    action = int(action)
    print(f"Action {translate_action[action]}, Points: {points}")
    if("End" not in translate_action[action]):
        plt.imshow(input_t.reshape((grid_size,)*2),
               interpolation='none', cmap='gray')
        display.display(plt.gcf())
    last_frame_time = set_max_fps(last_frame_time)
def set_max_fps(last_frame_time,FPS = 1):
    current_milli_time = lambda: int(round(time.time() * 1000))
    sleep_time = 1./FPS - (current_milli_time() - last_frame_time)
    if sleep_time > 0:
        time.sleep(sleep_time)
    return current_milli_time()
def test(model):
    global last_frame_time
    plt.ion()
    # Define environment, game
    env = Catch(grid_size)
    c = 0
    last_frame_time = 0
    points = 0
    for e in range(10):
        loss = 0.
        env.reset()
        game_over = False
        # get initial input
        input_t = env.observe()
        display_screen(3,points,input_t)
        c += 1
        while not game_over:
            input_tm1 = input_t
            # get next action
            q = model.predict(input_tm1)
            action = int(np.argmax(q[0]))
            # apply action, get rewards and new state
            input_t, reward, game_over = env.act(action)
            points += reward
            display_screen(action,points,input_t)
            c += 1
    display_screen(4,points,input_t)

In [3]:
train(model)

NameError: name 'train' is not defined

In [ ]:
test(model)